Full names: Noam Laub (id 209700897, Gal Israeli (id 318865813)
kaggle notebook: https://www.kaggle.com/code/noamlaub/notebook2b040e6681

In [1]:
#Prasad Shelke - Project Price of House Pridiction
#importing lab

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
# sklearn imports
from sklearn import metrics
from sklearn import pipeline
from sklearn import linear_model
from sklearn import preprocessing
from sklearn import neural_network
from sklearn import model_selection
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from IPython.display import Image
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [3]:
train_data = pd.read_csv("../input/house-prices-advanced-regression-techniques/train.csv")
test_data = pd.read_csv("../input/house-prices-advanced-regression-techniques/test.csv")

ANALYZING THE DATA

we need to check if there are any nulls and we need to remove or add data to them that will not change the statistics of the dataset

In [4]:
print(train_data.isna().any())
print(test_data.isna().any())
train_data.info()

Id               False
MSSubClass       False
MSZoning         False
LotFrontage       True
LotArea          False
                 ...  
MoSold           False
YrSold           False
SaleType         False
SaleCondition    False
SalePrice        False
Length: 81, dtype: bool
Id               False
MSSubClass       False
MSZoning          True
LotFrontage       True
LotArea          False
                 ...  
MiscVal          False
MoSold           False
YrSold           False
SaleType          True
SaleCondition    False
Length: 80, dtype: bool
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-nu

In [5]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1455 non-null   object 
 3   LotFrontage    1232 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   object 
 6   Alley          107 non-null    object 
 7   LotShape       1459 non-null   object 
 8   LandContour    1459 non-null   object 
 9   Utilities      1457 non-null   object 
 10  LotConfig      1459 non-null   object 
 11  LandSlope      1459 non-null   object 
 12  Neighborhood   1459 non-null   object 
 13  Condition1     1459 non-null   object 
 14  Condition2     1459 non-null   object 
 15  BldgType       1459 non-null   object 
 16  HouseStyle     1459 non-null   object 
 17  OverallQual    1459 non-null   int64  
 18  OverallC

After anaylizing the features types and null counts, we decided to remove 4 features because nthey contain too many nulls (Alley, PoolQC, Fence, MiscFeature).
For the rest of the features containing nulls we will add a random value for features of type Object, and add the average for features of Int/Float type.

In [6]:
# fill empty values in the dataframe
def fill_na_mean(df, column_name):
    df_not_null = df[~df[column_name].isnull()]
    df[column_name].fillna(df_not_null[column_name].mean(), inplace=True) 

def fill_na_random_pick_column_distribution(df, column_name):
    df_not_null = df[~df[column_name].isnull()]
    # use lambda func to choose randomly from
    df[column_name] = df[column_name].apply(lambda x: np.random.choice(df_not_null[column_name]) if pd.isnull(x) else x)

In [7]:
#Adding average to numerical features containing nulls
fill_na_mean(train_data, 'LotFrontage')
fill_na_mean(train_data, 'MasVnrArea')
fill_na_mean(train_data, 'GarageYrBlt')

#Adding random values to categorial features containing nulls
fill_na_random_pick_column_distribution(train_data,'MasVnrType')
fill_na_random_pick_column_distribution(train_data,'BsmtQual')
fill_na_random_pick_column_distribution(train_data,'FireplaceQu')
fill_na_random_pick_column_distribution(train_data,'BsmtCond')
fill_na_random_pick_column_distribution(train_data,'BsmtExposure')
fill_na_random_pick_column_distribution(train_data,'BsmtFinType1')
fill_na_random_pick_column_distribution(train_data,'BsmtFinType2')
fill_na_random_pick_column_distribution(train_data,'Electrical')
fill_na_random_pick_column_distribution(train_data,'GarageType')
fill_na_random_pick_column_distribution(train_data,'GarageFinish')
fill_na_random_pick_column_distribution(train_data,'GarageQual')
fill_na_random_pick_column_distribution(train_data,'GarageCond')

#Deleting features containing too many nulls
train_data = train_data.drop(['Alley'], axis=1) 
train_data = train_data.drop(['PoolQC'], axis=1)
train_data = train_data.drop(['Fence'], axis=1)
train_data = train_data.drop(['MiscFeature'], axis=1)
train_data = train_data.drop(['Id'], axis=1)

/tmp/ipykernel_18/3942538685.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column_name].fillna(df_not_null[column_name].mean(), inplace=True)


In [8]:
# determine categorical and numerical features
train_labels = train_data['SalePrice'].copy()
train_cor = train_data.copy()
train_data = train_data.drop(['SalePrice'], axis=1)
all_cols = np.array(train_data.columns)
all_cols_cor = np.array(train_cor.columns)

numerical_cols = train_data.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = train_data.select_dtypes(include=['object', 'bool']).columns

print('Numerical Cols:', numerical_cols)
print('Categorical Cols:', categorical_cols)

Numerical Cols: Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
       'MoSold', 'YrSold'],
      dtype='object')
Categorical Cols: Index(['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heati

In [9]:
import plotly.graph_objects as go
# calculate score and loss from cv (KFold or LPO) and display graphs
def get_cv_score_and_loss(X, t, model, transformer=None,
                          k=None,
                          p=None,
                          show_score_loss_graphs=False):
  
    scores_losses_df = pd.DataFrame(columns=['fold_id', 'split', 'score', 'loss'])

    if k is not None:
        cv = KFold(n_splits=k, shuffle=True,random_state=42)
    elif p is not None:
        cv = LeavePOut(p)
    else:
        raise ValueError('you need to specify k or p in order for the cv to work')

    for i, (train_ids, val_ids) in enumerate(cv.split(X)):
        # from index to record assignment
        X_train = X.loc[train_ids]
        t_train = t.loc[train_ids]
        X_val = X.loc[val_ids]
        t_val = t.loc[val_ids]

        # fit and predict
        model.fit(X_train, t_train)
        y_train = model.predict(X_train)
        y_val = model.predict(X_val)
        
        # assign results into scores df
        scores_losses_df.loc[len(scores_losses_df)] = [i, 'train', model.score(X_train, t_train),
                                                       mean_squared_error(t_train, y_train)]
        scores_losses_df.loc[len(scores_losses_df)] =[i, 'val', model.score(X_val, t_val),
                                                      mean_squared_error(t_val, y_val)]

    #split into two dfs , val and train
    val_scores_losses_df = scores_losses_df[scores_losses_df['split']=='val']
    train_scores_losses_df = scores_losses_df[scores_losses_df['split']=='train']

    #calculate mean score and loss for train and validation across all models
    mean_val_score = val_scores_losses_df['score'].mean()
    mean_val_loss = val_scores_losses_df['loss'].mean()
    mean_train_score = train_scores_losses_df['score'].mean()
    mean_train_loss = train_scores_losses_df['loss'].mean()

    if show_score_loss_graphs:
      fig = px.line(scores_losses_df, x='fold_id', y='score', color='split', title=f'Mean Val Score: {mean_val_score:.2f}, Mean Train Score: {mean_train_score:.2f}')
      fig.show()
      fig = px.line(scores_losses_df, x='fold_id', y='loss', color='split', title=f'Mean Val Loss: {mean_val_loss:.2f}, Mean Train Loss: {mean_train_loss:.2f}')
      fig.show()
      display(scores_losses_df)


    return mean_val_score, mean_val_loss, mean_train_score, mean_train_loss

Here we are analyzing the best K for the K fold cross validation

In [10]:
# use column transformer to insert different transformers for each column


ct = ColumnTransformer([
                        ("encoding", OneHotEncoder(sparse_output=False, handle_unknown='ignore'),categorical_cols),
                        ("standard", preprocessing.StandardScaler(), numerical_cols)]
    )
model_pipe = make_pipeline(ct, SGDRegressor(random_state=1))
val_score, val_loss, train_score, train_loss  = get_cv_score_and_loss(train_data,
                                                                     train_labels,
                                                                     model_pipe, 
                                                                     transformer=ct,
                                                                     k=10,
                                                                     p=None,
                                                                     show_score_loss_graphs=True)

print(f'mean cv val score: {val_score:.2f}\nmean cv val loss {val_loss:.2f}')
print(f'mean cv val score: {train_score:.2f}\nmean cv val loss {train_loss:.2f}')

/opt/conda/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  sf: grouped.get_group(s if len(s) > 1 else s[0])


/opt/conda/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



,fold_id,split,score,loss
0,0,train,0.887478,6.742201e+08
1,0,val,0.868748,1.199203e+09
2,1,train,0.887065,7.134345e+08
3,1,val,0.872194,7.887593e+08
4,2,train,0.875228,7.968786e+08
5,2,val,0.880718,6.662417e+08
6,3,train,0.885083,7.022280e+08
7,3,val,0.837307,1.299320e+09
8,4,train,0.893986,6.827164e+08
9,4,val,0.694788,1.554607e+09


mean cv val score: 0.82
mean cv val loss 1130836607.11
mean cv val score: 0.88
mean cv val loss 731246965.22


We decided to take Kfold with a k of 2, as the loss on the training data and validation was the closest to each other at k=2.

We will now analyze the best Polynomial degree for the numerical features 

In [11]:
# show graph of score and loss by plynomial degree of numerical features
def show_degree_graphs_cv_train(X, t, model, k=None, p=None, max_degree=10):
    
    val_train_score_loss_df = pd.DataFrame(columns=['degree', 'split',
                                                    'score', 'loss'])
    for i in range(1, max_degree):
        ct_enc_std_poly = ColumnTransformer([
                                              ("encoding", OneHotEncoder(sparse_output=False, handle_unknown='ignore'),
                                              categorical_cols),
                                              ("standard_poly",
                                              make_pipeline(PolynomialFeatures(degree=i),preprocessing.StandardScaler()),
                                               numerical_cols)])
        
        model_pipe = make_pipeline(ct_enc_std_poly, model)
        
        val_score, val_loss, train_score, train_loss = get_cv_score_and_loss(X,
                                                                             t,
                                                                             model_pipe,
                                                                             transformer=ct,
                                                                             k=k,
                                                                             p=p,
                                                                             show_score_loss_graphs=False
                                                                             )
        
        val_train_score_loss_df.loc[len(val_train_score_loss_df)] = [i, 'train', train_score, train_loss]
        
        val_train_score_loss_df.loc[len(val_train_score_loss_df)] = [i, 'cv', val_score, val_loss]

    fig = px.line(val_train_score_loss_df, x='degree', y='score', color='split')
    fig.show()
    fig = px.line(val_train_score_loss_df, x='degree', y='loss', color='split')
    fig.show()

show_degree_graphs_cv_train(train_data,
                            train_labels,
                            SGDRegressor(random_state=1),
                            k=2,
                            max_degree=5)

/opt/conda/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



/opt/conda/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



We chose degree of 1 because the loss on the train and on the cross validation  was the closest at that degreee of polynom.

next we will choose the categories we will use

In [12]:
# find best subset of features on this dataset
from sklearn.feature_selection import RFECV
from sklearn.model_selection import RepeatedKFold


all_cols = categorical_cols.tolist() + numerical_cols.tolist()

ct_enc_std = ColumnTransformer([
            ("encoding", OrdinalEncoder(), categorical_cols),
            ("standard", preprocessing.StandardScaler(), numerical_cols)])

X_encoded = pd.DataFrame(ct_enc_std.fit_transform(train_data, train_labels), columns=all_cols)

selector = RFECV(
    SGDRegressor(random_state=1),
    cv=RepeatedKFold(n_splits=2, n_repeats=10, random_state=42)
    ).fit(X_encoded, train_labels)

display(X_encoded.loc[:, selector.support_])

fig = go.Figure()
fig.add_trace(go.Scatter(x=[i for i in range(1, len(selector.feature_names_in_) + 1)]
                         , y=selector.cv_results_['mean_test_score']))
fig.update_xaxes(title_text="Number of features selected")
fig.update_yaxes(title_text="Cross validation R2 score")
fig.show()

,Street,LandContour,LandSlope,Condition2,RoofStyle,RoofMatl,MasVnrType,ExterQual,ExterCond,Foundation,...,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,FullBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageCars
0,1.0,3.0,0.0,2.0,1.0,1.0,1.0,2.0,4.0,2.0,...,-0.459303,-0.793434,1.161852,0.370333,0.789741,0.163779,-0.211454,0.912210,-0.951226,0.311725
1,1.0,3.0,0.0,2.0,1.0,1.0,1.0,3.0,4.0,1.0,...,0.466465,0.257140,-0.795163,-0.482512,0.789741,0.163779,-0.211454,-0.318683,0.600495,0.311725
2,1.0,3.0,0.0,2.0,1.0,1.0,1.0,2.0,4.0,2.0,...,-0.313369,-0.627826,1.189351,0.515013,0.789741,0.163779,-0.211454,-0.318683,0.600495,0.311725
3,1.0,3.0,0.0,2.0,1.0,1.0,1.0,3.0,4.0,0.0,...,-0.687324,-0.521734,0.937276,0.383659,-1.026041,0.163779,-0.211454,0.296763,0.600495,1.650307
4,1.0,3.0,0.0,2.0,1.0,1.0,1.0,2.0,4.0,2.0,...,0.199680,-0.045611,1.617877,1.299326,0.789741,1.390023,-0.211454,1.527656,0.600495,1.650307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1.0,3.0,0.0,2.0,1.0,1.0,2.0,3.0,4.0,2.0,...,-0.238122,-0.542435,0.795198,0.250402,0.789741,0.163779,-0.211454,0.296763,0.600495,0.311725
1456,1.0,3.0,0.0,2.0,1.0,1.0,2.0,3.0,4.0,1.0,...,1.104925,2.355701,-0.795163,1.061367,0.789741,0.163779,-0.211454,0.296763,2.152216,0.311725
1457,1.0,3.0,0.0,2.0,1.0,1.0,1.0,0.0,2.0,4.0,...,0.215641,0.065656,1.844744,1.569647,0.789741,1.390023,-0.211454,1.527656,2.152216,-1.026858
1458,1.0,3.0,0.0,2.0,3.0,1.0,1.0,3.0,4.0,1.0,...,0.046905,-0.218982,-0.795163,-0.832788,-1.026041,-1.062465,-0.211454,-0.934130,-0.951226,-1.026858


We can see that the best score we received with 33 features, now we want to check which features we should remove.

In [13]:
print(selector.feature_names_in_)
print(selector.ranking_)
print(selector.support_)
selector.get_feature_names_out()

['MSZoning' 'Street' 'LotShape' 'LandContour' 'Utilities' 'LotConfig'
 'LandSlope' 'Neighborhood' 'Condition1' 'Condition2' 'BldgType'
 'HouseStyle' 'RoofStyle' 'RoofMatl' 'Exterior1st' 'Exterior2nd'
 'MasVnrType' 'ExterQual' 'ExterCond' 'Foundation' 'BsmtQual' 'BsmtCond'
 'BsmtExposure' 'BsmtFinType1' 'BsmtFinType2' 'Heating' 'HeatingQC'
 'CentralAir' 'Electrical' 'KitchenQual' 'Functional' 'FireplaceQu'
 'GarageType' 'GarageFinish' 'GarageQual' 'GarageCond' 'PavedDrive'
 'SaleType' 'SaleCondition' 'MSSubClass' 'LotFrontage' 'LotArea'
 'OverallQual' 'OverallCond' 'YearBuilt' 'YearRemodAdd' 'MasVnrArea'
 'BsmtFinSF1' 'BsmtFinSF2' 'BsmtUnfSF' 'TotalBsmtSF' '1stFlrSF' '2ndFlrSF'
 'LowQualFinSF' 'GrLivArea' 'BsmtFullBath' 'BsmtHalfBath' 'FullBath'
 'HalfBath' 'BedroomAbvGr' 'KitchenAbvGr' 'TotRmsAbvGrd' 'Fireplaces'
 'GarageYrBlt' 'GarageCars' 'GarageArea' 'WoodDeckSF' 'OpenPorchSF'
 'EnclosedPorch' '3SsnPorch' 'ScreenPorch' 'PoolArea' 'MiscVal' 'MoSold'
 'YrSold']
[17  1 34  1 19 10  1 2

array(['Street', 'LandContour', 'LandSlope', 'Condition2', 'RoofStyle',
       'RoofMatl', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType2', 'Heating',
       'CentralAir', 'GarageQual', 'GarageCond', 'PavedDrive',
       'SaleCondition', 'LotArea', 'OverallQual', 'OverallCond',
       'BsmtFinSF1', 'BsmtFinSF2', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'GrLivArea', 'FullBath', 'BedroomAbvGr', 'KitchenAbvGr',
       'TotRmsAbvGrd', 'Fireplaces', 'GarageCars'], dtype=object)

In [14]:
train_data = train_data.filter(['Street', 'LandContour', 'LandSlope', 'Condition2', 'RoofStyle',
       'RoofMatl', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType2',
       'CentralAir', 'Functional', 'GarageQual', 'GarageCond',
       'PavedDrive', 'SaleCondition', 'LotArea', 'OverallCond',
       'YearRemodAdd', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
       'GrLivArea', 'BsmtFullBath', 'FullBath', 'BedroomAbvGr',
       'Fireplaces', 'GarageCars', 'WoodDeckSF', 'ScreenPorch'])

In [15]:
test_data = test_data.filter(['Street', 'LandContour', 'LandSlope', 'Condition2', 'RoofStyle',
       'RoofMatl', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType2',
       'CentralAir', 'Functional', 'GarageQual', 'GarageCond',
       'PavedDrive', 'SaleCondition', 'LotArea', 'OverallCond',
       'YearRemodAdd', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
       'GrLivArea', 'BsmtFullBath', 'FullBath', 'BedroomAbvGr',
       'Fireplaces', 'GarageCars', 'WoodDeckSF', 'ScreenPorch'])

In [16]:
train_cor = train_cor.filter(['SalePrice','Street', 'LandContour', 'LandSlope', 'Condition2', 'RoofStyle',
       'RoofMatl', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType2',
       'CentralAir', 'Functional', 'GarageQual', 'GarageCond',
       'PavedDrive', 'SaleCondition', 'LotArea', 'OverallCond',
       'YearRemodAdd', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
       'GrLivArea', 'BsmtFullBath', 'FullBath', 'BedroomAbvGr',
       'Fireplaces', 'GarageCars', 'WoodDeckSF', 'ScreenPorch'])

In [17]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 34 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Street         1459 non-null   object 
 1   LandContour    1459 non-null   object 
 2   LandSlope      1459 non-null   object 
 3   Condition2     1459 non-null   object 
 4   RoofStyle      1459 non-null   object 
 5   RoofMatl       1459 non-null   object 
 6   MasVnrType     565 non-null    object 
 7   ExterQual      1459 non-null   object 
 8   ExterCond      1459 non-null   object 
 9   Foundation     1459 non-null   object 
 10  BsmtQual       1415 non-null   object 
 11  BsmtCond       1414 non-null   object 
 12  BsmtExposure   1415 non-null   object 
 13  BsmtFinType2   1417 non-null   object 
 14  CentralAir     1459 non-null   object 
 15  Functional     1457 non-null   object 
 16  GarageQual     1381 non-null   object 
 17  GarageCond     1381 non-null   object 
 18  PavedDri

In [18]:
fill_na_mean(test_data, 'GarageCars')
fill_na_mean(test_data, 'BsmtFullBath')
fill_na_mean(test_data, 'TotalBsmtSF')
fill_na_mean(test_data, 'BsmtFinSF2')
fill_na_mean(test_data, 'BsmtUnfSF')
fill_na_random_pick_column_distribution(test_data,'MasVnrType')
fill_na_random_pick_column_distribution(test_data,'BsmtQual')
fill_na_random_pick_column_distribution(test_data,'BsmtCond')
fill_na_random_pick_column_distribution(test_data,'BsmtExposure')
fill_na_random_pick_column_distribution(test_data,'GarageCond')
fill_na_random_pick_column_distribution(test_data,'GarageQual')
fill_na_random_pick_column_distribution(test_data,'Functional')
fill_na_random_pick_column_distribution(test_data,'BsmtFinType2')

/tmp/ipykernel_18/3942538685.py:4: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





Differentiating between the categorial and numerical features

In [19]:
numerical_cols = train_data.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = train_data.select_dtypes(include=['object', 'bool']).columns

print('Numerical Cols:', numerical_cols)
print('Categorical Cols:', categorical_cols)

Numerical Cols: Index(['LotArea', 'OverallCond', 'YearRemodAdd', 'BsmtFinSF2', 'BsmtUnfSF',
       'TotalBsmtSF', 'GrLivArea', 'BsmtFullBath', 'FullBath', 'BedroomAbvGr',
       'Fireplaces', 'GarageCars', 'WoodDeckSF', 'ScreenPorch'],
      dtype='object')
Categorical Cols: Index(['Street', 'LandContour', 'LandSlope', 'Condition2', 'RoofStyle',
       'RoofMatl', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType2', 'CentralAir',
       'Functional', 'GarageQual', 'GarageCond', 'PavedDrive',
       'SaleCondition'],
      dtype='object')


In [20]:
cor_categorical_cols = train_cor.select_dtypes(include=['object', 'bool']).columns
cor_numerical_cols = train_cor.select_dtypes(include=['int64', 'float64']).columns

In [21]:
ct = ColumnTransformer([
                        ("encoding", OneHotEncoder(sparse_output=False, handle_unknown='ignore'),categorical_cols)])

In [22]:
# print lasso, ridge and elasticnet scores as regression 
from sklearn.model_selection import cross_val_score

sgd_lasso_reg = SGDRegressor(penalty='l1',
                             random_state=1)
sgd_ridge_reg = SGDRegressor(penalty='l2',
                             random_state=1)
sgd_elastic_reg = SGDRegressor(penalty='elasticnet',
                               random_state=1)

print("R2 score for regression:")
print('sgd_lasso',
      cross_val_score(make_pipeline(ct,preprocessing.StandardScaler(), sgd_lasso_reg), train_data, train_labels, cv=2).mean())

print('sgd_ridge',
      cross_val_score(
          make_pipeline(ct,
              preprocessing.StandardScaler(), 
              sgd_ridge_reg), 
              train_data, train_labels, cv=2).mean())

print('sgd_elastic',
      cross_val_score(
          make_pipeline(ct,
              preprocessing.StandardScaler(), 
              sgd_elastic_reg), 
             train_data, train_labels, cv=2).mean())

R2 score for regression:
sgd_lasso 0.27098000555206214
sgd_ridge 0.27150989355113264
sgd_elastic 0.2714304657235824


In [23]:
# create accuracy score for ordinal predictions
from sklearn.metrics import make_scorer, accuracy_score

def get_accurate_ordinal_preds_from_numeric_preds(
    preds, min=None, max=None):
    if min is None:
        min = round(min(preds))
    if max is None:
        max = round(max(preds))
    preds = np.asarray(preds).ravel()
    return np.array(
        [round(p) if min <= p and p <= max else min if
         p < min else max for p in preds])

def accuracy_for_ordinal(y, y_pred):
    min_ord = min(y)
    max_ord = max(y)
    y_pred_ord = get_accurate_ordinal_preds_from_numeric_preds(y_pred, min=min_ord, max=max_ord)
    return accuracy_score(y, y_pred_ord)

print("Accuracy score for regression:")

print('sgd_lasso', 
      cross_val_score(make_pipeline(ct,preprocessing.StandardScaler(), sgd_lasso_reg), 
                      train_data, train_labels, cv=2, 
                      scoring=make_scorer(accuracy_for_ordinal)).mean())

print('sgd_ridge', 
      cross_val_score(make_pipeline(ct,preprocessing.StandardScaler(), sgd_ridge_reg), 
                      train_data, train_labels, cv=2, 
                      scoring=make_scorer(accuracy_for_ordinal)).mean())

print('sgd_elastic', 
      cross_val_score(make_pipeline(ct,preprocessing.StandardScaler(), sgd_elastic_reg), 
                      train_data, train_labels, cv=2, 
                      scoring=make_scorer(accuracy_for_ordinal)).mean())

Accuracy score for regression:
sgd_lasso 0.0
sgd_ridge 0.0
sgd_elastic 0.0


We will not choose a regulator as the accuracy of the model drops significally when using one.

In [24]:
all_cols = np.array(train_data.columns)
all_cols_cor = np.array(train_cor.columns)

In [25]:
ct_enc_std = ColumnTransformer([
            ("encoding", OrdinalEncoder(), cor_categorical_cols),
            ("standard", preprocessing.StandardScaler(), cor_numerical_cols)])
train_cor = pd.DataFrame(ct_enc_std.fit_transform(train_cor, train_labels), columns=all_cols_cor)

In [26]:
plt.figure(figsize=(100,100))
cor = np.abs(train_cor.corr())
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds, vmin=0, vmax=1)
plt.show()

In [27]:
common_columns = train_data.columns.intersection(test_data.columns)

df1_filtered = train_data[common_columns]
df2_filtered = test_data[common_columns]
df1_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 34 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Street         1460 non-null   object
 1   LandContour    1460 non-null   object
 2   LandSlope      1460 non-null   object
 3   Condition2     1460 non-null   object
 4   RoofStyle      1460 non-null   object
 5   RoofMatl       1460 non-null   object
 6   MasVnrType     1460 non-null   object
 7   ExterQual      1460 non-null   object
 8   ExterCond      1460 non-null   object
 9   Foundation     1460 non-null   object
 10  BsmtQual       1460 non-null   object
 11  BsmtCond       1460 non-null   object
 12  BsmtExposure   1460 non-null   object
 13  BsmtFinType2   1460 non-null   object
 14  CentralAir     1460 non-null   object
 15  Functional     1460 non-null   object
 16  GarageQual     1460 non-null   object
 17  GarageCond     1460 non-null   object
 18  PavedDrive     1460 non-null

In [28]:
ct = ColumnTransformer([
    ("encoding", OrdinalEncoder(), categorical_cols),
    ("standard", MinMaxScaler(), numerical_cols)])

X_encoded = pd.DataFrame(ct.fit_transform(df1_filtered, train_labels), columns=all_cols)
test_encoded = pd.DataFrame(ct.fit_transform(df2_filtered), columns=all_cols)

# Create the selected model to predict the test
model = RFECV(SGDRegressor(max_iter=10000,random_state=1), cv=RepeatedKFold(n_splits=5, n_repeats=10, random_state=1)).fit(X_encoded, train_labels)

In [29]:
# train with grid search and get best parameters
from sklearn.model_selection import GridSearchCV

hyper_parameters = {'penalty': ('l2', 'l1', 'elasticnet'), 'alpha': [0.0001, 0.001, 0.01, 0.1], 'eta0': [0.001, 0.01, 0.1, 0.5]}

gs_model = GridSearchCV(
    SGDClassifier(random_state=1),
     hyper_parameters).fit(X_encoded, train_labels)

print('Accuracy score for classification:')
print('gs_model', gs_model.best_score_)
print('best params', gs_model.best_params_)

/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning:

The least populated class in y has only 1 members, which is less than n_splits=5.



Accuracy score for classification:
gs_model 0.00684931506849315
best params {'alpha': 0.01, 'eta0': 0.001, 'penalty': 'l1'}


In [30]:
display(test_data)

,Street,LandContour,LandSlope,Condition2,RoofStyle,RoofMatl,MasVnrType,ExterQual,ExterCond,Foundation,...,BsmtUnfSF,TotalBsmtSF,GrLivArea,BsmtFullBath,FullBath,BedroomAbvGr,Fireplaces,GarageCars,WoodDeckSF,ScreenPorch
0,Pave,Lvl,Gtl,Norm,Gable,CompShg,BrkFace,TA,TA,CBlock,...,270.0,882.0,896,0.0,1,2,0,1.0,140,120
1,Pave,Lvl,Gtl,Norm,Hip,CompShg,BrkFace,TA,TA,CBlock,...,406.0,1329.0,1329,0.0,1,3,0,1.0,393,0
2,Pave,Lvl,Gtl,Norm,Gable,CompShg,Stone,TA,TA,PConc,...,137.0,928.0,1629,0.0,2,3,1,2.0,212,0
3,Pave,Lvl,Gtl,Norm,Gable,CompShg,BrkFace,TA,TA,PConc,...,324.0,926.0,1604,0.0,2,3,1,2.0,360,0
4,Pave,HLS,Gtl,Norm,Gable,CompShg,BrkFace,Gd,TA,PConc,...,1017.0,1280.0,1280,0.0,2,2,0,2.0,0,144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,Pave,Lvl,Gtl,Norm,Gable,CompShg,BrkFace,TA,TA,CBlock,...,546.0,546.0,1092,0.0,1,3,0,0.0,0,0
1455,Pave,Lvl,Gtl,Norm,Gable,CompShg,BrkFace,TA,TA,CBlock,...,294.0,546.0,1092,0.0,1,3,0,1.0,0,0
1456,Pave,Lvl,Gtl,Norm,Gable,CompShg,BrkFace,TA,TA,CBlock,...,0.0,1224.0,1224,1.0,1,4,1,2.0,474,0
1457,Pave,Lvl,Gtl,Norm,Gable,CompShg,BrkFace,TA,TA,PConc,...,575.0,912.0,970,0.0,1,3,0,0.0,80,0


In [31]:
# predict the test with SGDClassifier

# create the submission file
submission = pd.read_csv("../input/house-prices-advanced-regression-techniques/sample_submission.csv")
submission['SalePrice'] = model.predict(test_encoded)
submission.to_csv('submission.csv', index=False)
display(submission)

,Id,SalePrice
0,1461,96677.560995
1,1462,122674.254467
2,1463,181989.575151
3,1464,170011.784325
4,1465,180812.013747
...,...,...
1454,2915,78718.389898
1455,2916,84685.728640
1456,2917,188854.384787
1457,2918,100201.810610
